In [1]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
from utility import *
from hmmlearn import hmm
import seaborn as sns
import pickle
import matplotlib.cm as cm
import networkx as nx
%matplotlib inline

modelFolder='/Users/yyu/Documents/Psychology/insight/EEG_data/results/model_cv/'
dataFolder='/Users/yyu/Documents/Psychology/insight/EEG_data/processed/cra/'

In [2]:
featureList=["{:.1f},{:.1f}".format(c,f) for c in includeSources for f in includeFreqs]

# Make split

In [20]:
#split into 5 fold for cross validation
nfold=5
splitdf=pd.DataFrame()
for subid in os.listdir(dataFolder):
    if not os.path.isdir(dataFolder+subid):continue
    trialdict={}
    for type in ['ANA','INS','TO']:
        trialdict[type]=[]
        for f in glob.glob(dataFolder+subid+'*/*_'+type+'_*'):
            filename=(f.split('/')[-1])
            trialdict[type].append(filename.split('.')[0].split('_')[-1])
        split=np.array_split(trialdict[type], nfold)
        for i in range(nfold):
            splitdf=pd.concat([splitdf,pd.DataFrame({'id':subid,'type':type,'fold':i,'trialID':split[i]})])

In [21]:
splitdf.to_csv(dataFolder+'split.csv',index=None)

# Test

In [3]:
splitdf=pd.read_csv(dataFolder+'split.csv')
splitdf.head()

,id,type,fold,trialID
0,104,ANA,0,17
1,104,ANA,0,16
2,104,ANA,0,14
3,104,ANA,0,15
4,104,ANA,1,11


In [8]:
total=0
nstate='12'
for file in ['20_iter_30_0',
            '120_iter_30_1',
            '220_iter_30_2',
            '320_iter_30_3',
            '420_iter_30_4']:
    foldIndex=int(file[-1])
    #with initialization
    with open(modelFolder+nstate+'_full_'+file+'.pkl', "rb") as file: 
        model=pickle.load(file) 
  
    print(np.round(model.monitor_.history))

    nlist=[]
    featureDf=pd.DataFrame()
    for idFolder in os.listdir(dataFolder):
        if not os.path.isdir(dataFolder+idFolder):continue
        for type in ['ANA','INS','TO']:
            excludeList=splitdf[(splitdf['id']==int(idFolder))&(splitdf['type']==type)&(splitdf['fold']==foldIndex)]['trialID'].values
            for f in glob.glob(dataFolder+idFolder+'*/*_'+type+'_*'):
                if f.split('.')[1]!='csv':continue
                filename = (f.split('/')[-1])
                if not (int(filename.split('.')[0].split('_')[-1]) in excludeList): continue
                tmp = pd.read_csv(f, index_col=None)
                nlist.append(len(tmp))
                featureDf=pd.concat([featureDf,tmp[featureList]])
    tmp=model.score_samples(featureDf[featureList].values, lengths=nlist)
    print(foldIndex,len(featureDf),tmp[0])
    total=total+tmp[0]
    

[-1.50905428e+08 -1.50902502e+08]
0 200485 -40538653.70654826
[-1.52220673e+08 -1.52218948e+08]
1 193821 -39220234.81376308
[-1.52704059e+08 -1.52695864e+08]
2 191371 -38707781.05925246
[-1.53759791e+08 -1.53753787e+08]
3 186286 -37667354.103037745
[-1.55735918e+08 -1.55734228e+08]
4 176542 -35704117.11251


In [9]:
total

-191838140.79511154

In [12]:
len(featureDf)

200485

In [23]:
model.score_samples(featureDf[featureList].values, lengths=nlist)

(-36467871.83082484,
 array([[1.64743759e-03, 1.40686473e-07, 9.96597602e-01, 8.36572280e-05,
         1.67116228e-03],
        [1.00783147e-04, 7.35832835e-10, 9.99583005e-01, 3.24750187e-06,
         3.12963658e-04],
        [4.01027630e-04, 3.81614526e-09, 9.78558376e-01, 7.86676247e-05,
         2.09619245e-02],
        ...,
        [1.97316916e-02, 6.84476523e-06, 2.96327726e-03, 5.64694030e-04,
         9.76733492e-01],
        [4.17605646e-02, 2.15943222e-04, 5.30112618e-03, 3.30837221e-03,
         9.49413994e-01],
        [5.13718997e-02, 8.60763987e-04, 9.68025536e-03, 1.67981470e-02,
         9.21288934e-01]]))

In [25]:
foldIndex=1

#with initialization
with open(modelFolder+'5_full_1_iter_10_'+str(foldIndex)+'.pkl', "rb") as file: 
    model=pickle.load(file)
print(np.round(model.monitor_.history))


[-1.37173763e+08 -1.37171958e+08]


In [26]:
model.score_samples(featureDf[featureList].values, lengths=nlist)

(-36432058.993409045,
 array([[2.97578247e-03, 1.00645352e-03, 9.95987667e-01, 1.27860668e-07,
         2.99689876e-05],
        [7.97505387e-05, 2.29472128e-04, 9.99689011e-01, 9.95160849e-10,
         1.76544700e-06],
        [2.40794620e-04, 2.49620065e-02, 9.74760076e-01, 3.76787070e-09,
         3.71190303e-05],
        ...,
        [1.63968906e-03, 9.98070724e-01, 2.07251883e-04, 3.30258718e-06,
         7.90325420e-05],
        [5.80604151e-03, 9.92993726e-01, 4.42278300e-04, 1.75738538e-04,
         5.82215875e-04],
        [1.30162406e-02, 9.76488256e-01, 2.60087536e-03, 7.68866279e-04,
         7.12576171e-03]]))